# 1. loading the dataset

In [6]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import pipeline

In [7]:
dataset_path = 'dataset.json'
# Open and load the JSON data
with open(dataset_path, 'r') as f:
    data = json.load(f)

In [8]:
df = pd.DataFrame(data)
df['label_numeric'] = df['label'].apply(lambda x: 1 if x == 'positive' else 0)
print("\nFirst 5 rows of the DataFrame:")
print(df.head())


First 5 rows of the DataFrame:
                                           Sentence     label  label_numeric
0            He broke the bank and lost everything.  negative              0
1  She broke the bank with her amazing performance.  positive              1
2            The mouse in my kitchen ruined my day.  negative              0
3    The new mouse for my computer works perfectly.  positive              1
4                   He left the party feeling blue.  negative              0


In [9]:
challenge_data = [
    {"Sentence": "I hate the selfishness in you", "label": "negative"},
    {"Sentence": "I hate any one who can hurt you", "label": "positive"}
]
challenge_df = pd.DataFrame(challenge_data)
challenge_df['label_numeric'] = challenge_df['label'].apply(lambda x: 1 if x == 'positive' else 0)

In [10]:
train_df, init_test_df = train_test_split(
    df,
    test_size=0.3,
    random_state=42,
    stratify=df['label_numeric']
)

print(f"\nSplit main data into {len(train_df)} training and {len(init_test_df)} initial test examples.")

test_df = pd.concat([init_test_df, challenge_df], ignore_index=True)

print(f"Final test set created with {len(test_df)} examples.")


Split main data into 42 training and 18 initial test examples.
Final test set created with 20 examples.


In [11]:
X_train = train_df['Sentence']
y_train = train_df['label_numeric']
X_test = test_df['Sentence']
y_test = test_df['label_numeric']


# 1. The Baseline:GloVe Embeddings + classifier

In [7]:
# Download the Pre-trained GloVe Embeddings
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-10-03 20:51:30--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-10-03 20:51:31--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2025-10-03 20:54:10 (5.17 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [12]:
# Load the GloVe Vectors into

glove_path = 'glove.6B.100d.txt'
embedding_dim = 100

# map words to their vector representations
embedding_dict = {}


with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_dict[word] = vector

In [13]:
def sentence_to_vector(sentence, embedding_dict, dim=100):
    """
    Converts a sentence to its average GloVe vector representation.
    """
    # Split the sentence into words (tokens)
    words = sentence.lower().split()

    # Initialize an empty list to store the vectors of words found in the dictionary
    word_vectors = []

    for word in words:
        if word in embedding_dict:
            word_vectors.append(embedding_dict[word])

    # If no words in the sentence are in our GloVe dictionary, return a zero vector
    if not word_vectors:
        return np.zeros(dim)

    # Calculate the mean of the word vectors to get the sentence vector
    sentence_vector = np.mean(word_vectors, axis=0)

    return sentence_vector


X_train_glove = np.array([sentence_to_vector(s, embedding_dict) for s in X_train])
X_test_glove = np.array([sentence_to_vector(s, embedding_dict) for s in X_test])

print("\nConverted all sentences to GloVe vectors.")
print(f"Shape of training vectors: {X_train_glove.shape}")
print(f"Shape of testing vectors: {X_test_glove.shape}")


Converted all sentences to GloVe vectors.
Shape of training vectors: (42, 100)
Shape of testing vectors: (20, 100)


In [14]:

# Initialize the classifier
glove_classifier = LogisticRegression(random_state=42)

print("\nTraining the Logistic Regression classifier on GloVe vector")

# Train the model
glove_classifier.fit(X_train_glove, y_train)

print("Training complete.")


Training the Logistic Regression classifier on GloVe vector
Training complete.


In [15]:
print("\n--- Evaluating the GloVe Baseline Model ---")

# Make predictions on the full test set
y_pred_glove = glove_classifier.predict(X_test_glove)

print("\nClassification Report (Overall Performance on Test Set):")
print("-" * 60)
print(classification_report(y_test, y_pred_glove, target_names=['negative', 'positive']))


results_df = pd.DataFrame({
    'Sentence': X_test,
    'True_Label': y_test.map({0: 'negative', 1: 'positive'}),
    'Predicted_Label': pd.Series(y_pred_glove, index=X_test.index).map({0: 'negative', 1: 'positive'})
})
results_df['Correct'] = results_df['True_Label'] == results_df['Predicted_Label']
# evaluate on the challenge sentences mentiones in the task
challenge_sentence_1 = "I hate the selfishness in you"
challenge_sentence_2 = "I hate any one who can hurt you"
challenge_results_df = results_df[
    results_df['Sentence'].isin([challenge_sentence_1, challenge_sentence_2])
]

print("\n\nAnalysis of Performance on Core Challenge Sentences:")
print("-" * 60)

pd.set_option('display.max_colwidth', None)

if challenge_results_df.empty:
    print("WARNING: The challenge sentences were not found in the test set.")
else:
    print(challenge_results_df)


--- Evaluating the GloVe Baseline Model ---

Classification Report (Overall Performance on Test Set):
------------------------------------------------------------
              precision    recall  f1-score   support

    negative       0.67      1.00      0.80        10
    positive       1.00      0.50      0.67        10

    accuracy                           0.75        20
   macro avg       0.83      0.75      0.73        20
weighted avg       0.83      0.75      0.73        20



Analysis of Performance on Core Challenge Sentences:
------------------------------------------------------------
                           Sentence True_Label Predicted_Label  Correct
18    I hate the selfishness in you   negative        negative     True
19  I hate any one who can hurt you   positive        negative    False


## BERT contextual Solution

In [16]:
models_to_test = {
    "BERT": "distilbert-base-uncased-finetuned-sst-2-english",
    "DistilBERT": "distilbert-base-uncased-finetuned-sst-2-english",
    "RoBERTa (Twitter)": "cardiffnlp/twitter-roberta-base-sentiment-latest",
    "RoBERTa (Sentiment)": "siebert/sentiment-roberta-large-english"
}

In [18]:
loaded_pipelines = {}

print("="*60)
print("LOADING ALL SENTIMENT ANALYSIS MODELS...")
print("="*60)

# 3. Loop through the models and load each one
for model_name, model_checkpoint in models_to_test.items():
    print(f"\nLoading model: {model_name}...")
    try:
        # Load the pipeline and store it in our dictionary
        loaded_pipelines[model_name] = pipeline(
            "sentiment-analysis",
            model=model_checkpoint,
            device=0  # Use GPU if available
        )
        print(f"--> Successfully loaded '{model_name}'.")
    except Exception as e:
        print(f"--> FAILED to load model {model_checkpoint}. Error: {e}")

LOADING ALL SENTIMENT ANALYSIS MODELS...

Loading model: BERT...


Device set to use cpu


--> Successfully loaded 'BERT'.

Loading model: DistilBERT...


Device set to use cpu


--> Successfully loaded 'DistilBERT'.

Loading model: RoBERTa (Twitter)...


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


--> Successfully loaded 'RoBERTa (Sentiment)'.


In [19]:
challenge_sentence_1 = "I hate the selfishness in you"
challenge_sentence_2 = "I hate any one who can hurt you"

# Convert the test data to a list for the pipeline
test_sentences = X_test.tolist()

# 2. Loop through each pre-loaded model and evaluate it
for model_name, sentiment_analyzer in loaded_pipelines.items():
    print("\n" + "="*80)
    print(f"EVALUATING MODEL: {model_name}")
    print("="*80)

    # A. Run inference on the entire test set
    print(f"Running inference on {len(test_sentences)} test sentences...")
    predictions_raw = sentiment_analyzer(test_sentences)
    print("Inference complete.")

    # B. Process the predictions to a consistent format
    predicted_labels = []
    for pred in predictions_raw:
        label_norm = pred['label'].lower()
        if not (label_norm == 'positive' or label_norm == 'negative'):
            if label_norm in ['label_2', 'label_1', 'pos']:
                 label_norm = 'positive'
            else:
                 label_norm = 'negative'
        predicted_labels.append(label_norm)

    predicted_numeric = [1 if label == 'positive' else 0 for label in predicted_labels]

    # C. Calculate and display overall performance metrics
    accuracy = accuracy_score(y_test, predicted_numeric)
    report = classification_report(y_test, predicted_numeric, target_names=['negative', 'positive'])

    print(f"\n--- Overall Performance for {model_name} ---")
    print(f"Accuracy: {accuracy:.2%}")
    print("Classification Report:")
    print(report)

    # D. Analyze and display performance on the two core challenge sentences
    results_df = pd.DataFrame({
        'Sentence': X_test,
        'True_Label': y_test.map({0: 'negative', 1: 'positive'}),
        'Prediction': predicted_labels
    })

    challenge_results = results_df[
        results_df['Sentence'].isin([challenge_sentence_1, challenge_sentence_2])
    ]

    print(f"\n--- Analysis of {model_name} on Core Challenge Sentences ---")
    if challenge_results.empty:
        print("WARNING: The challenge sentences were not found in the test set.")
    else:
        # Add a correctness column for clarity in this specific output
        challenge_results['Correct'] = challenge_results['True_Label'] == challenge_results['Prediction']
        print(challenge_results.to_string())


EVALUATING MODEL: BERT
Running inference on 20 test sentences...
Inference complete.

--- Overall Performance for BERT ---
Accuracy: 95.00%
Classification Report:
              precision    recall  f1-score   support

    negative       0.91      1.00      0.95        10
    positive       1.00      0.90      0.95        10

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.95      0.95      0.95        20


--- Analysis of BERT on Core Challenge Sentences ---
                           Sentence True_Label Prediction  Correct
18    I hate the selfishness in you   negative   negative     True
19  I hate any one who can hurt you   positive   negative    False

EVALUATING MODEL: DistilBERT
Running inference on 20 test sentences...


/tmp/ipython-input-1574393188.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  challenge_results['Correct'] = challenge_results['True_Label'] == challenge_results['Prediction']


Inference complete.

--- Overall Performance for DistilBERT ---
Accuracy: 95.00%
Classification Report:
              precision    recall  f1-score   support

    negative       0.91      1.00      0.95        10
    positive       1.00      0.90      0.95        10

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.95      0.95      0.95        20


--- Analysis of DistilBERT on Core Challenge Sentences ---
                           Sentence True_Label Prediction  Correct
18    I hate the selfishness in you   negative   negative     True
19  I hate any one who can hurt you   positive   negative    False

EVALUATING MODEL: RoBERTa (Twitter)
Running inference on 20 test sentences...


/tmp/ipython-input-1574393188.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  challenge_results['Correct'] = challenge_results['True_Label'] == challenge_results['Prediction']


Inference complete.

--- Overall Performance for RoBERTa (Twitter) ---
Accuracy: 85.00%
Classification Report:
              precision    recall  f1-score   support

    negative       0.77      1.00      0.87        10
    positive       1.00      0.70      0.82        10

    accuracy                           0.85        20
   macro avg       0.88      0.85      0.85        20
weighted avg       0.88      0.85      0.85        20


--- Analysis of RoBERTa (Twitter) on Core Challenge Sentences ---
                           Sentence True_Label Prediction  Correct
18    I hate the selfishness in you   negative   negative     True
19  I hate any one who can hurt you   positive   negative    False

EVALUATING MODEL: RoBERTa (Sentiment)
Running inference on 20 test sentences...


/tmp/ipython-input-1574393188.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  challenge_results['Correct'] = challenge_results['True_Label'] == challenge_results['Prediction']


Inference complete.

--- Overall Performance for RoBERTa (Sentiment) ---
Accuracy: 95.00%
Classification Report:
              precision    recall  f1-score   support

    negative       0.91      1.00      0.95        10
    positive       1.00      0.90      0.95        10

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.95      0.95      0.95        20


--- Analysis of RoBERTa (Sentiment) on Core Challenge Sentences ---
                           Sentence True_Label Prediction  Correct
18    I hate the selfishness in you   negative   negative     True
19  I hate any one who can hurt you   positive   negative    False


/tmp/ipython-input-1574393188.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  challenge_results['Correct'] = challenge_results['True_Label'] == challenge_results['Prediction']


In [20]:
sentences_to_analyze = [
    "I hate any one who can hurt you", # Baseline
    "I hate the selfishness in you", # Other core challenge
    "I love you and I hate any one who can hurt you", # Strong Emotional Anchor
    "you are my friend I hate any one who can hurt you", # Relational Anchor
    "Here is my opinion: I hate any one who can hurt you", # Neutral Framing
    "It is a kind sentiment that I hate any one who can hurt you" # Descriptive Anchor
]

In [21]:

all_results = []

print("="*80)
print("RUNNING ANALYSIS ON AUGMENTED SENTENCES ACROSS ALL LOADED MODELS")
print("="*80)

# loop through the PRE-LOADED models and test each sentence
for model_name, sentiment_analyzer in loaded_pipelines.items():
    print(f"\nTesting with model: {model_name}")

    # Run inference on the entire list of sentences at once for efficiency
    predictions_raw = sentiment_analyzer(sentences_to_analyze)

    # Process each result
    for sentence, result_raw in zip(sentences_to_analyze, predictions_raw):
        # Normalize the label to a consistent 'positive' or 'negative'
        label_norm = result_raw['label'].lower()
        if not (label_norm == 'positive' or label_norm == 'negative'):
            if label_norm in ['label_2', 'label_1', 'pos']:
                 label_norm = 'positive'
            else:
                 label_norm = 'negative'

        score = result_raw['score']

        # Store the detailed result
        all_results.append({
            'Model': model_name,
            'Sentence': sentence,
            'Prediction': label_norm,
            'Confidence': f"{score:.2%}"
        })

# display the Final, Comprehensive Summary Table
print("\n" + "="*80)
print("FINAL SUMMARY: IMPACT OF DIFFERENT CONTEXTUAL ANCHORS ACROSS MODELS")
print("="*80)

summary_df = pd.DataFrame(all_results)
summary_df['Display_Result'] = summary_df['Prediction'] + " (" + summary_df['Confidence'] + ")"

final_pivot = summary_df.pivot_table(
    index='Sentence',
    columns='Model',
    values='Display_Result',
    aggfunc='first'
)

# Define a logical order for columns and rows
column_order = [name for name in models_to_test.keys() if name in final_pivot.columns]
final_pivot = final_pivot.reindex(columns=column_order)
final_pivot = final_pivot.reindex(sentences_to_analyze)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 120)

print(final_pivot)

RUNNING ANALYSIS ON AUGMENTED SENTENCES ACROSS ALL LOADED MODELS

Testing with model: BERT

Testing with model: DistilBERT

Testing with model: RoBERTa (Twitter)

Testing with model: RoBERTa (Sentiment)

FINAL SUMMARY: IMPACT OF DIFFERENT CONTEXTUAL ANCHORS ACROSS MODELS
Model                                                                     BERT         DistilBERT  RoBERTa (Twitter)  \
Sentence                                                                                                               
I hate any one who can hurt you                              negative (98.15%)  negative (98.15%)  negative (92.47%)   
I hate the selfishness in you                                negative (99.51%)  negative (99.51%)  negative (93.66%)   
I love you and I hate any one who can hurt you               positive (99.92%)  positive (99.92%)  positive (63.12%)   
you are my friend I hate any one who can hurt you            positive (99.84%)  positive (99.84%)  positive (48.90%)   
Here is 

In [23]:
print("\n--- Evaluating the GloVe Baseline Model ---")

# Convert the standard test sentences to GloVe vectors
X_test_glove = np.array([sentence_to_vector(s, embedding_dict) for s in X_test])
# Make predictions
y_pred_glove = glove_classifier.predict(X_test_glove)

print("\nClassification Report (Overall Performance on Standard Test Set):")
print("-" * 60)
print(classification_report(y_test, y_pred_glove, target_names=['negative', 'positive']))


print("\n\nAnalysis of GloVe Performance on Challenge & Augmented Sentences:")
print("-" * 60)

# Create a list to store the results for these specific sentences
glove_specific_results = []

for sentence in sentences_to_analyze:
    # Convert the single sentence to its GloVe vector
    sentence_vec = sentence_to_vector(sentence, embedding_dict).reshape(1, -1)

    # Get the model's prediction (0 or 1)
    prediction_numeric = glove_classifier.predict(sentence_vec)[0]

    # Convert the numeric prediction to a string label
    prediction_label = "positive" if prediction_numeric == 1 else "negative"

    glove_specific_results.append({
        'Sentence': sentence,
        'GloVe_Prediction': prediction_label
    })

# Convert the results to a DataFrame for clean printing
glove_analysis_df = pd.DataFrame(glove_specific_results)

pd.set_option('display.max_colwidth', None)
print(glove_analysis_df.to_string())


--- Evaluating the GloVe Baseline Model ---

Classification Report (Overall Performance on Standard Test Set):
------------------------------------------------------------
              precision    recall  f1-score   support

    negative       0.67      1.00      0.80        10
    positive       1.00      0.50      0.67        10

    accuracy                           0.75        20
   macro avg       0.83      0.75      0.73        20
weighted avg       0.83      0.75      0.73        20



Analysis of GloVe Performance on Challenge & Augmented Sentences:
------------------------------------------------------------
                                                      Sentence GloVe_Prediction
0                              I hate any one who can hurt you         negative
1                                I hate the selfishness in you         negative
2               I love you and I hate any one who can hurt you         negative
3            you are my friend I hate any one who c